# 04. Correlating Spike Trains

* 싸이그래머 / 싸이ML : 파이뉴로 [1]
* 김무성

<img src="figures/cap4.1.png" width=600>

## The canonical data analysis cascade ( five principal steps in code)

As far as we can tell, all sensory systems (with the exception of olfaction, which is special) follow the following  ve steps, and there are principled reasons for this (Fig. 4.1).
* <b>Step 1: Transduction</b>
    - Every sensory system has to convert some kind of physical energy in the environment into the common currency of the brain.
    - The code equivalent of this is to write dedicated “loader” code.
        - Its purpose is to load data from whatever format it was logged in—each physiological data recording system creates its own data  les, e.g., .plx  les, .nev  les, .nex  les, .xls  les, and .txt  les are some popular formats. For you to be able to do anything with these in POM, they have to be converted into a format that POM can use  rst.

* <b>Step 2: Filtering</b>
    - Once a spike train reaches the cortex, it will be processed. 
    - Of course this raises the issue of how the thalamus knows what irrelevant information is before it has been analyzed by the cortex.
    - The brain solves this in several ways, including an analysis for low-level salient features, such as fast motion or high contrast, and then feeds that information back to the thalamus—there are many recurrent feedback loops to  ne-tune the  ltering process.
    - We strongly recommend to implement this step in code, for similar reasons.
    -  This step can be called “cleaning,” “pruning,” or “filtering” of the data.
* <b>Step 3: Formatting</b>
    - The next step performed by the visual system is a categorical analysis of the data arriving from the thalamus. 
        - This step is performed by the early visual system, particularly V1. Here, the visual system determines the location and basic orientation of line segments (Hubel & Wiesel, 2004), and starts the process of determining what is foreground and what is background  gure (Craft, Schütze, Niebur, & Von Der Heydt, 2007). 
    -  Heuristically, this step can be understood as setting the stage for further analysis, not so much doing a lot of in depth analysis here already.
    - Thus, we conceive of this step as “formatting” the data for further analysis.
    - It is an absolutely critical step for data analysis.
        - Once data are formatted properly, the rest of the analysis is usually rather straightforward.
        - It might be unsettling to the beginner, but is not unusual to spend *most* of one’s time writing analysis code in this step, simply “formatting” the data.
        - Once data structures are set up properly, the actual analysis often corresponds to something very simple, like “loop through all participants in these conditions, then compare their means.”
* <b>Step 4: Computations</b>
    - In the visual system, this step is implemented by extrastriate cortex—the cortical regions after striate cortex (or primary visual cortex) in the visual processing stream.
    - Interestingly, whereas the previous steps have been done mostly in serial fashion, one after the other (the feedback to thalamus notwithstanding), this step is better referred to as steps (plural) because they <font color="red">happen in parallel</font>, meaning that the signal might split into two or more copies so that multiple processes can occur on it simultaneously (Wallisch & Movshon, 2008).
    - We recommend to do something similar in code to implement this step. Specifically, <font color="red">we recommend to create as many parallel analysis streams</font> as there are analysis goals.
* <b>Step 5: Output</b>
    - This might come as a surprise to people living in the modern age, but the purpose of the visual system is not to provide fancy images for one’s viewing pleasure, but to improve the survivability of the organism. 
        - This is true for sensory systems in general. Perception is not an end in itself—unless it results in motor output, the outcomes of its calculations are irrelevant. 
        - Over time, the system has been optimized to provide more adaptive outputs. 
        - In primates, <font color="red">the end result of the visual processing cascade in extrastriate areas is linked to motor cortex</font> in order to transform the visual input to real-world outputs and memory systems to store information (the results of the computations on the inputs). 
    - We will do the same here, in code. Specifically, we will hook up the outputs of step 4, e.g., 4a, 4b to corresponding outputs, i.e., 5a, 5b. This analogy is tight. 
    - There are three principal kinds of outputs from the computation steps
        - Sometimes, we just want to output some numbers to the screen. 
        - Sometimes, the output will be a figure that visualizes the results of a computation. 
        - Usually, we also want to store the results so that we can load them in later without having to redo the entire analysis from scratch (this is particularly important as some analyses can take rather long).
* In addition to these five steps implemented by sensory systems of the brain, we recommend adding a zeroth and a sixth step. 
    * The zeroth step is an <b>“initialization” step</b> 
    * Finally, in the spirit of making the code maintainable, and even runable a couple of months after writing it, we recommend writing a  le that corresponds to the sixth step, which is kind of a “wrapper” and <b>“readme”  file</b> that (if you wrote a  le for each of the  ve steps) calls the right  les with the right parameters in the right order and some kind of documentation of what is going on.

<img src="figures/cap4.2.png" width=600 />

## Snyder, Morais, Willis, and Smith (2015)
* Snyder, A. C., Morais, M. J., Willis, C. M., & Smith, M. A. (2015). Global network in uences on local functional connectivity. Nature Neuroscience, 18(5), 736–743.

Without further ado, let’s jump into a full-blown data analysis project. Speci cally, we will recreate some of the analy- ses from Snyder, Morais, Willis, and Smith (2015) from scratch. This paper attempts to answer the question whether the spiking of individual neurons is in uenced by the activity of the network that they are embedded in. To do so, Snyder et al. recorded signals from a visual area with a 96 electrode array, splitting the signal from each electrode into a spike channel and a channel of analog signals, or local  eld potentials (LFPs) while the animals were shown oriented gratings. For the purposes of this chapter, we focus on the information from the spike channels. We will address the analysis of analog signals in frequency space in Chapter 5, Analog Signals.

Thus, we will use 96 time series of spiking information as source data in this chapter. We already encountered one time series in Chapter 2, From 0 to 0.01 and Chapter 3, Wrangling Spikes Trains. As a general rule, much of the data in neuroscience will come in the form of time series. Prominent examples would be EEG, MEG, EMG, LFP, spike trains, participant responses, and the like, so knowing how to handle them is important.


(구글 번역 :
더 이상 고민하지 않고 본격적인 데이터 분석 프로젝트에 뛰어 들어 봅시다. 구체적으로 Snyder, Morais, Willis, Smith (2015)의 분석을 처음부터 다시 만들 것입니다. 이 논문은 개인 뉴런의 스파이크가 그들이 포함되어있는 네트워크의 활동에 의해 영향을 받는지 여부에 대한 질문에 대답하려고 시도한다. 그렇게하기 위해서 Snyder et al. 동물들이 배향 된 격자를 보여 주면서 각 전극으로부터의 신호를 스파이크 채널 및 아날로그 신호의 채널, 또는 로컬 엘 드 전위 (LFP)로 분할하는 96 전극 배열을 갖는 시각 영역으로부터의 기록 된 신호. 이 장의 목적을 위해 우리는 스파이크 채널의 정보에 중점을 둡니다. 5 장, 아날로그 신호에서 주파수 공간에서의 아날로그 신호 분석을 다룰 것입니다.

따라서이 장에서는 원본 데이터로 96 시간 계열의 스파이 킹 정보를 사용합니다. 우리는 이미 2 장에서 하나의 시계열을 보았습니다. 0에서 0.01까지 그리고 3 장, 스파이크 열차를 논하십시오. 일반적으로 신경 과학의 많은 데이터는 시계열 형태로 나타날 것입니다. EEG, MEG, EMG, LFP, 스파이크 열차, 참여자 응답 등이 대표적인 예가 될 수 있으므로이를 처리하는 방법을 아는 것이 중요합니다.)

-----------------------

## spike count correlation

As we have more than one time series (96 in fact) we can now look at the relationship between time series from differ- ent electrodes in order to calculate a quantity that is called the “spike count correlation.” We will explain the concept of spike count correlations and why they matter in detail later, but let’s  rst build up to that. It is time to get started.

(구글 번역 :
사실 우리는 하나 이상의 시계열 (96 개)이 있기 때문에 "스파이크 수 상관 관계"라고 불리는 양을 계산하기 위해 서로 다른 전극의 시계열 간의 관계를 볼 수 있습니다. 스파이크 카운트 상관 관계 및 왜 나중에 세부적으로 중요할까요? 이제 시작할 시간입니다.)

### header 

In [1]:
#*************************************************************************************
#This program recreates the spike count correlation analysis from Snyder et al. (2015)
#It assumes that the data file from the recording array exists in the same
#directory as the analysis file.
#Name and email of code creator
#V1: 06/16/2016: Steps 0-3
#V2: 06/20/2016: Steps 4 and 5
#V3: 06/21/2016: Adding parameters to step 0
#*************************************************************************************

### step 0: Initialization

In [2]:
## 0. Python Init
# For Python, we load the functions that we’ll be using and assign
# some of them nicknames
import scipy.io
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sc
numChannels = 96
noiseCodes = [0,255]
timeBase=np.arange(-.2,2.5,.01)
gratingOn = 0
gratingOff = 2

### step 1, the loader

In [3]:
# Pseudocode
# 1: Loader. Load the data file

In [4]:
arrayDATA_data_path = "../data/arrayDATA.mat"

In [5]:
matIn=scipy.io.loadmat(arrayDATA_data_path)

In [6]:
print(list(matIn.keys()))

['__header__', '__version__', '__globals__', 'DATA']


In [7]:
matIn['DATA']

array([[ (array([[  20.        ,    1.        ,    0.62400001],
       [  20.        ,    1.        ,    0.63      ],
       [  20.        ,    3.        ,    0.65200001],
       [  20.        ,    3.        ,    0.65899998],
       [  20.        ,  255.        ,    0.68599999],
       [  20.        ,    1.        ,    0.68900001],
       [  20.        ,    1.        ,    0.71100003],
       [  20.        ,  255.        ,    0.73100001],
       [  20.        ,    3.        ,    0.74199998],
       [  20.        ,    3.        ,    0.75700003],
       [  20.        ,    2.        ,    0.764     ],
       [  20.        ,    4.        ,    0.77899998],
       [  20.        ,    2.        ,    0.78600001],
       [  20.        ,    3.        ,    0.81199998],
       [  20.        ,    1.        ,    0.82599998],
       [  20.        ,    3.        ,    0.84799999],
       [  20.        ,  255.        ,    0.85699999],
       [  20.        ,    4.        ,    0.866     ],
       [  20.     

That structure is called DATA. You can glean its properties by typing DATA in MATLAB (or matIn[‘DATA’] in Python. 
* If you do so, you’ll see that it contains 2300 rows with two fields: nev and ori. 
* Each row 
    - represents a trial. 
* ori 
    - represents the orientation of the grating for a given trial and
* nev 
    - is a matrix that represents the spiking information. 
    - Each trial contains its own nev matrix. 
    - Rows 
        - in the nev matrix represent individual spikes per trial, one for each spike during the trial. 
    - Column 1 
        - represents the electrode on which the spike was detected,
    - column 3 
        - represents the corresponding time since the beginning of the trial at which the spike was detected, and 
    - column 2 
        - represents a “sort code.” 
        - The sort code is a number from 0 to 255. 
        - Neural data, in particular data from electrode arrays, have to be carefully sorted before they can be processed further. 
        - The reason for this is that each electrode records the voltage at the tip at a given time, but this voltage is the result of the spiking activity of an unknown number of neurons in an unknown con guration around the electrode tip (Lewicki, 1998). 
        - Thus, spike sorting—resolving the individual sources that all contribute to the same voltage sum—is fundamentally an inverse problem and remains rather tricky, despite best attempts at solving it (Pillow, Shlens, Chichilnisky, & Simoncelli, 2013). 
        - Here, take solace in the fact that spike sorting has already been done, and done carefully. 
        - This is indicated by the sort codes assigned to each spike.
        - Sort codes 0 and 255 represent noise (accidental threshold crossings or threshold crossings for artificial reasons such as animal movement or instrument noise) and will need to be discarded from further analysis. 
        - Other sort codes –1 to 254 represent units (where one unit is thought to be nearly one neuron) that were assigned to different clusters. 
        - If a spike has the same sort code, it was discharged from the same neural cluster (either multiunit or the same single unit).

* Structures are nested (or hierarchical) and labeled matrices. 
* Each  field has 
    - a label (such as ori) and 
    - a content (could be matrices of numbers or characters).
* Structures can contain arbitrary numbers of levels. 
* Structures make for great storage units, but doing computations on them can be challenging, which is why we’ll need further processing steps (particularly step 3, formatting) of our canonical data analysis cascade. 
* Fields are accessed by typing a period or dot. 
* If you don’t remember the  eld names, you can tab-complete them. 
* For instance, 
    - to access the orientation of the 57th trial in MATLAB, 
        - type DATA(57).ori 
            - the output indicates that the orientation of the grating was zero degrees for that trial. 
    - If you want to know the spiking activity from trial 210, 
        - type DATA(210).nev 
            - to reveal that a brisk 1747 action potentials from a wide range of channels were discharged during that trial.

In [11]:
# Pseudocode
# Determine several parameters 
#   that will be used in later analysis from the data itself, 
#     such as 
#       the number of trials, 
#       the grating
#       orientation and 
#       number of grating orientations,

In [12]:
numTrials = len(matIn['DATA'])
allOris=[matIn['DATA'][_][0][1][0][0] for _ in range(numTrials)] #intermediate variable
ori = list(set(allOris))
numOri = len(ori)
trialIndices = defaultdict(list)

In [ ]:
for _ in range(numTrials) :
    print(matIn['DATA'][1])

In [14]:
numTrials

2300

In [13]:
numTrials

2300

In [15]:
allOris[:10]

[90.0, 0.0, 0.0, 90.0, 90.0, 0.0, 0.0, 90.0, 0.0, 90.0]

In [16]:
ori[:10]

[0.0, 90.0]

In [17]:
numOri

2

In [18]:
trialIndices

defaultdict(list, {})

* numTrials 
    - represents the number of trials and can be gotten simply by counting the number of rows of the DATA structure. 
* The variable ori 
    - represents the number of unique grating orientations used in the experiment. 
    - Here, it was two orientations, 0 and 90, 
        - but we want to write the code so that it is flexible enough to accommodate different orientations (and different numbers of orientations) in the future, should we receive them. 
        - To do so, 
            - we use the function <font color="blue">unique</font> in MATLAB, and <font color="blue">set</font> in Python. 
            - Each returns an ordered list of all unique orientation values. 
            - We provide this function with a vector of all orientations from the structure, as you can see in the code above.
* trialIndices 
    - will contain the trial indices in the DATA structure that correspond to a given grating orientation. 
    - Note that we allocate trialIndices as a cell array in MATLAB, and as a defaultdict (a special kind of dictionary that allows the user to append to values even if the keys do not yet exist) in Python.
    - The number of columns of this cell array corresponds to the number of unique orientations. 
    - The reason we want to allocate it as a dictionary or cell array in POM is because we want to allow for the possibility that not each orientation was presented an equal number of times (although this is the case here). 
    - For instance, 
        - if 
            - one orientation was presented 1200 times and 
            - the other 1300 times, and 
            - we represent this as resident matrices of a cell array,
            - this is no problem. 
        - If 
            - we represented it as a 1150 × 2 matrix (one row for each trial, different orientations in each column), 
            - we would have to assume that each orientation was presented exactly the same number of times. 
        - If 
            - this was off by a single number, 
            - we could not represent it as a matrix.
        - This is an unacceptable degree of rigidity, so in order to make our code more flexible, we represent these indices as a dictionary or cell array from the beginning.
    - For now, we are just preallocating them by declaring their structure. If you type trialIndices, you will see that there is nothing in the matrices, but the structure of the cell array itself—a 1 × 2 matrix of matrices has been established in MATLAB, and an empty dictionary in Python: defaultdict(list, {}).

### Step 2 + 3: Formatting of valid data (which implies the identi cation and elimination of invalid data)

In [ ]:
# Pseudocode
# 2+3: Pruning and Formatting
#     Step 1: Identify trial numbers (indices) 
#             that correspond to a given orientation

In [19]:
for tt in range(numTrials):
    for oo in ori:
        if allOris[tt] == oo:
            trialIndices[oo].append(tt)

In [21]:
list(trialIndices.keys())

[0.0, 90.0]

In [23]:
trialIndices[0.0][:10]

[1, 2, 5, 6, 8, 10, 13, 15, 16, 18]

In [ ]:
# Pseudocode
# 1. Find the indices of spikes that were recorded on 
#            a given electrode (channel), 
#               from a given trial and condition (grating orientation)
# 2. Use those indices to find spikes 
#            that were  agged as one kind of noise
# 3. Use them again to find spikes 
#            that were  agged as another kind of noise
# 4. Merge the two sets of noise into a big “killset”
# 5. Eliminate all indices 
#            that correspond to noise from the variable 
#                  that contains spike indices for 
#                     a given electrode(channel), trial, and condition
# 6. Find the corresponding spike times for 
#            the remaining valid spike indices and 
#    put them in the large storage structure, the spikeTimes cell array
# 7. Loop through all channels (electrodes)
# 8. Loop through all conditions (grating orientation types) 
# 9. Look through all trials of a given type

In [ ]:
# 코딩!!!



-------------------------------------------

# 다음에 계속

# 참고 
* [1] Neural Data Science: A Primer with MATLAB and Python - https://www.amazon.com/Neural-Data-Science-MATLAB®-PythonTM/dp/0128040432/
* [2] Chapter Files - https://www.elsevier.com/books-and-journals/book-companion/9780128040430/chapter-files#Chapter%20Files